In [1]:
%cd /workspace/flair
# !python -m spacy download en_core_web_lg

/workspace/flair


# elasticsearch

In [26]:
# !pip install elasticsearch elasticsearch_dsl

In [67]:
tk = "$1"
a = {tk: {tk: 1}}
a[tk][tk] += 1
a

{'$1': {'$1': 2}}

In [75]:
tickers = ["$1", "$2", "$3"]
keywords = ["aaa", "bbb", "ccc"]

ticker_name = {}
ticker_count = {}
keyword_count = {}

def add(tickers, keywords):
    for tk in tickers:
        if tk not in ticker_count:
            ticker_count[tk] = {"co_tickers": {}, "co_keywords": {}}
        for ctk in tickers:
            try:
                ticker_count[tk]["co_tickers"][ctk] += 1
            except KeyError:
                ticker_count[tk]["co_tickers"][ctk] = 1
        for ckw in keywords:
            try:
                ticker_count[tk]["co_keywords"][ckw] += 1
            except KeyError:
                ticker_count[tk]["co_keywords"][ckw] = 1

    for kw in keywords:
        if kw not in keyword_count:
            keyword_count[kw] = {"co_tickers": {}, "co_keywords": {}}
        for ctk in tickers:
            try:
                keyword_count[kw]["co_tickers"][ctk] += 1
            except KeyError:
                keyword_count[kw]["co_tickers"][ctk] = 1
        for ckw in keywords:
            try:
                keyword_count[kw]["co_keywords"][ckw] += 1
            except KeyError:
                keyword_count[kw]["co_keywords"][ckw] = 1

add(tickers, keywords)
add(tickers, keywords)

ticker_count
keyword_count

{'aaa': {'co_tickers': {'$1': 2, '$2': 2, '$3': 2},
  'co_keywords': {'aaa': 2, 'bbb': 2, 'ccc': 2}},
 'bbb': {'co_tickers': {'$1': 2, '$2': 2, '$3': 2},
  'co_keywords': {'aaa': 2, 'bbb': 2, 'ccc': 2}},
 'ccc': {'co_tickers': {'$1': 2, '$2': 2, '$3': 2},
  'co_keywords': {'aaa': 2, 'bbb': 2, 'ccc': 2}}}

In [47]:
%cd /workspace/flair/app/app
import json 
from pprint import pprint
import pandas as pd
from elasticsearch_dsl import Q
import es

es.init()

domain = "cnbc"

s = es.Page.search()
q = Q('wildcard', from_url=f'*{domain}*') & Q("term", http_status=200)

# data = {"text": [], "labels": []}
docs = []
labels = []
tickers = []

ticker_name = {}
ticker_count = {}
keyword_count = {}

def add(tickers, keywords):
    for tk in tickers:
        try:
            ticker_count[tk] = 
        except:
            ticker_count[tk] = 
        
    

for i, page in enumerate(s.filter(q).scan()):
    if i > 0:
        break

    try:
#         pprint(vars(page))
#         meta = json.loads(page.article_metadata)
#         pprint(meta)
        parsed = json.loads(page.parsed)
        pprint(parsed)
        
        _tickers = []
        for e in parsed["tickers"]:
            for label in e["labels"]:
                name, ticker = label
                try:
                    ticker_name[ticker].add(name)
                except:
                    ticker_name[ticker] = set([name])
                _tickers.append(ticker)
        print(_tickers)
        print(parsed["keywords"])
    except:
        pass

ticker_name

/workspace/flair/app/app
{'keywords': ['JPMorgan Chase & Co',
              'Amazon.com Inc',
              'Stock markets',
              'Ted Weschler',
              'Todd Combs',
              'Bank of America Corp',
              'Apple Inc',
              'American Express Co',
              'S&P 500 Index',
              'Warren Buffett',
              'Berkshire Hathaway Inc',
              'Markets',
              'Investment strategy',
              'business news'],
 'tickers': [{'labels': [['Berkshire Hathaway', 'BRK.A']],
              'text': 'Billionaire investor Warren Buffett told CNBC on Monday '
                      'that Berkshire Hathaway investment managers Ted '
                      'Weschler and Todd Combs have trailed the   by a "tiny '
                      'bit" since each joined the company.'},
             {'labels': [['American Express', 'AXP'],
                         ['Apple', 'AAPL'],
                         ['Bank of America', 'BAC']],
            

{'BRK.A': {'Berkshire Hathaway'},
 'AXP': {'American Express'},
 'AAPL': {'Apple'},
 'BAC': {'Bank of America'},
 'AMZN': {'Amazon'},
 'JPM': {'J.P. Morgan'}}

In [63]:
set(labels)

{'Aerospace and defense industry',
 'Airlines',
 'Alibaba Group Holding Ltd',
 'Alphabet Class A',
 'Amazon.com Inc',
 'Asia Economy',
 'Asia News',
 'Bank of America Corp',
 'Banks',
 'Boeing Co',
 'Breaking News: Business',
 'Breaking News: Markets',
 'Breaking News: Technology',
 'Business',
 'CNBC: Stock Market & Business',
 'Enterprise',
 'Europe News',
 'Finance',
 'Germany',
 'Housing',
 'Investment strategy',
 'LLC',
 'Life',
 'Markets',
 'Microsoft Corp',
 'NBCUniversal Media',
 'News',
 'Oracle Corp',
 'Personal finance',
 'Personal loans',
 'Personal saving',
 'Politics',
 'Real estate',
 'SAP AG',
 'SVMK Inc',
 'Slack Technologies Inc',
 'Software',
 'Southwest Airlines Co',
 'Special Reports',
 'Start-ups',
 'Stock markets',
 'Technology',
 'Transportation',
 'Travel',
 'Wall Street',
 'app',
 'app store',
 'appstore',
 'business news',
 'ios apps',
 'ipad',
 'iphone',
 'ipod touch',
 'itouch',
 'itunes'}

In [14]:
# !conda install -c conda-forge -y spacy 
# !pip install -U spacy
# !python -m spacy download zh_core_web_sm
!pip install git+https://github.com/LIAAD/yake

  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-9v41ayvi
  Running command git clone -q https://github.com/LIAAD/yake /tmp/pip-req-build-9v41ayvi
     |████████████████████████████████| 90 kB 1.2 MB/s eta 0:00:011
  Created wheel for yake: filename=yake-0.4.3-py2.py3-none-any.whl size=67215 sha256=68f7fd17891fd6f875e2a860135af97c2999746d5ba1591e40e7d13128c04b49
  Stored in directory: /tmp/pip-ephem-wheel-cache-j1qhthdw/wheels/52/79/f4/dae9309f60266aa3767a4381405002b6f2955fbcf038d804da
Successfully built yake


In [25]:
text = """
没人会否认，平井一夫就是是索粉与索尼相互养成的偶像。

成都魅力赏现场，索尼新一波创纪录的业绩预期每公布一个数字，在场的粉丝总会“Wow”一声。这声“Wow”在一个人出现时到达了最大声。并不难猜，这个人就是平井一夫。


实际上，从这个月一日起，平井一夫就不再担任索尼的总裁兼 CEO。他的新职位是退居幕后的索尼董事长。尽管如此，不管他是何种身份，也无论是粉丝之夜还是魅力赏，哪怕就是场外的体验展台，只要平井一夫现身，现场立马就会响起日语的、英语的、中文的叫喊声，躁动和哭腔。

整个场面并不比大牌明星见面会逊色。

一个日本企业家为什么在中国会获得如此高的知名度和号召力？套用到目前流行的明星养成选秀，这似乎也不难理解。


时间退回六年前，那时的索尼正处于风雨飘摇的末路时期。产品创新的乏力、资本市场的背弃，“破产”并非只是一句简单的调侃。

2012 年 4 月 1 日，平井一夫正式出任索尼公司 CEO 兼总裁。这时，PS 业务出身的他对于游戏铁粉之外的大多数索尼粉丝而言，只是一个素人。

他在任内推行的“One Sony 政策”和“中期计划”则是一场旷日持久的养成直播。

尽管难说尽善尽美，但在他的治下，一方面，索尼各个业务开始了产品线的精简；另一方面，在平井一夫的关照下，索尼成立了一个名为为 Future Lab （未来实验室）的创意研发项目。这项目的主要目的是向公众公开某些产品的原型，通过收集公众的想法意见来规划产品的发展方向。简单地说，该项目旨在让索尼那些黑科技原型机更能符合公众需求。


如此之下，索尼开始焕发新的生机。到平井一夫改革初见成效的 2015 财年，按通用会计准则计，索尼在这一财年共实现了 81057 亿日元（717.32 亿美元）的营收，营业利润为 2942 亿日元（约合 26.04 亿美元），净利润为 1478 亿日元（约合 13.08 亿美元），而此前一年，索尼净利润为-1260 亿日元。

在平井一夫即将离任的消息前后，索尼发布了 2017 财年 Q3 财报：销售额增长 11.5%，为 26723 亿日元。营业利润大涨 279.8%，达 3508 亿日元。所有业务（作者注：包括移动业务）均实现盈利，索尼 2017 财年前三季度累计盈利已达 7127 亿日元。并且，索尼在财报中将本财年营业利润预期再次提高，至 7200 亿日元。不出意外，索尼的利润在 2017 财年将创下 72 年历史的新高。

索粉们也从此放心——有平井一夫在，索尼说什么也不可能破产。

一个企业家偶像就此养成。

在索粉们心目中，平井一夫早已是超越一般 CEO 的存在。粉丝们一方面做着有关于他的表情包，调侃地追问“索尼今天破产了吗？”，另一方面，在他发布会上的“Wow”和“Kando（感动）”感召下，心甘情愿地献上膝盖和钱包，借着平井一夫（的表情包），粉丝和索尼保持着一种微妙的平衡。


如果说以上都是索粉们的自发行为，那么这场成都举行的索粉之夜上，索尼官方安排的节目《川剧变脸》中，演员最后一刻将脸变成“姨夫的微笑”便大有“官方逼死同人”之感。

看到这样严肃认真又带头恶搞的索尼，铁粉的钱包基本是保不住了。


而一个会沙画的索粉在画出平井一夫怀抱熊猫的画面并呈现在大屏幕上，而“姨夫”在一旁魔性微笑，是这场索粉之夜最让人动容的时刻，因为没有人知道这是不是这个力挽索尼于破产边缘的男人亲自参加的最后一场索粉之夜了。

对于索粉而言，他们一年之内最重要的节日是 Sony Expo

当然，这个活动还有一个简洁好记又十分贴切的名字——索尼魅力赏。从 2014 年开始，到 2017 年，这个活动先后在上海、北京、广州举办。


于索尼而言，魅力赏是索尼中国、索尼移动、索尼互娱、索尼音乐、索尼影视等索尼在华企业看家本领和顶级“黑科技”的大汇演，是对之前财年的完整总结，也是对崭新一年的美好展望。

2018 年 4 月 12 日，成都接过上海、北京、广州的衣钵，成为这些索尼在华企业的最新汇演舞台。

而对于索尼的中国铁杆粉丝们来说，魅力赏则意味着一场庆典。某种意义上，因为魅力赏开幕当晚的“索粉之夜”的存在，魅力赏又不止于节日。它成为了索粉们一个“绝对立场”被打开与志同道合的伙伴合作补完的过程，是一次索粉们对于人生方向的自我审视。


索尼魅力赏，还是索粉们的一场朝圣。

既然是朝圣，这就意味着其门槛是不低的。虽不至于“三步一跪、九步一叩”，但魅力赏，尤其是参加索粉之夜的几十个索粉们都是从全国成千上万的报名索粉中选出来的。说这群粉丝是精挑细选出来的坚定信仰者，并不为过。

让索粉们扬眉吐气的魅力赏，从索尼的业绩猛涨开始

从 2003 年 4 月已经成为专有名词的“索尼震撼（Sony Shock）”起，索尼的多项业务、乃至整个公司都陷入了低迷状态。

哪怕经过斯金格可以称为“巨变”的改革，从 2008 财年到 2014 财年，除 2012 财年外，索尼每一个财年都在亏损，其背后的原因与索尼在和韩国竞争对手的争夺中，为了市场占有率和营收而放弃利润有关，当然也和大地震等天灾有关。而 2012 财年的盈利则是因为索尼在当年出售了大楼等资产。

和如今日渐成为产业的“偶像饭圈”逻辑相似，当偶像的事业进入低谷，摇摆的粉丝会脱粉，坚定的铁粉会憋着一口气进行更大力度的应（花）援（钱）。

几十年持续的高质量消费电子品产出，和最近十年因为产品之外的因素而被看衰，这种独特的经历让索尼积累一批数量庞大又舍得花钱的铁粉。嗯，无论是对于品牌的深刻理解还是购买力，那些新晋的国产手机品牌粉丝比起索粉，只能算是后辈。

整个行业忠诚度和购买力能与索粉相对比的是苹果的果粉。当两者又有极大的不同，至少，憋着一口气的索粉总是勇于一边自黑、一边应援。这个圈子里的知名博主“今天 SONY 破产了吗 ”，看似戏谑的背后，其实质是一个一直用着索尼手机的粉丝。

话又说回去，即使每天都在进行“破产没有？”的灵魂拷问，索粉们最乐于见到的依然是索尼恢复往日荣耀。

足以让这些粉丝们扬眉吐气的是，索尼在本次魅力赏上带来了业绩和产品的好消息。

索尼中国总裁高桥洋在活动的一开始就公布：

集团全年有望创造 7200 亿日元营业利润的历史最佳业绩。此外，仅在 2017 年上半年，索尼在中国市场的业绩同比增长超过 30%。从由电视、数码影像、音频组成的消费电子业务，到 PlayStation 业务，再到索尼音乐等业务都可以说在各自领域取得了优势。
这里再说几个数字。

索尼电视在 2017 年 OLED 彩电市场的销售份额（金额）为 34%，排名第一、“65 英寸及以上市场份额（金额）排名第一”、“一万元人民币以上彩电市场份额（数量）排名第一”。

索尼游戏业务正式回归中国三年，共计发行了 150 多款游戏作品。截止 2017 年 12 月 31 日，PS4 全球销量超过 7360 万台，游戏作品软件销量为 6 亿 4500 万张，PlayStation Plus 会员突破 3150 万人。PS VR 全球销量达到 200 万台，出现了超过 150 款精品 PS VR 游戏，全球 PS VR 游戏销量超过 1220 万张。


世界上 TOP 6 的智能手机公司均是索尼影像传感器的客户。在介绍这一部分时，高桥洋还曝出了金句，“现在用索尼手机的人不多，但不用索尼传感器的手机很少”。索尼移动不姓索，算是官宣了（手动斜眼）。


如果说，这些数字太抽象的话，一个更具象的变化在说明索尼的财政状况确实在不断向好：相比去前年魅力赏逼仄的场地，今年，整个成都可谓都被索尼承包。从春熙路到来福士广场，这些成都的商业地标都充满了索尼的印记。

后记

不出意外，一份创造索尼历史新高的财报将在一个月之后正式发布，而它的分量索尼和索粉都明白。

厂家拿出优秀的产品，消费者实打实地花钱去购买。

什么是“你只管认真，我们帮你赢”？

这就是。
"""

In [37]:
from spacy.lang.zh import Chinese
import yake

nlp = Chinese()
doc = nlp(text)
t = " ".join([tk.text for tk in doc])


# text = "蘋果 公司 正 考量 用 一 億元 買下 英國 的 新創 公司 ， 加上 一個 標點 。 "

extractor = yake.KeywordExtractor(lan="zh", n=1)
keywords = extractor.extract_keywords(t)

In [38]:
keywords

[('亿日元', 0.0026444410756110803),
 ('索粉们', 0.002976195195791563),
 ('sony', 0.00671506498927978),
 ('wow', 0.00852361357827606),
 ('ceo', 0.009200188800847072),
 ('亿美元', 0.013942824262447678),
 ('另一方面', 0.0252324717162599),
 ('不出意外', 0.02604320100545872),
 ('如果说', 0.02604320100545872),
 ('市场份额', 0.02604320100545872),
 ('净利润', 0.02645257793868751),
 ('扬眉吐气', 0.02645257793868751),
 ('一口气', 0.02645257793868751),
 ('传感器', 0.02645257793868751),
 ('无论是', 0.026864608904568106),
 ('企业家', 0.026864608904568106),
 ('一方面', 0.026864608904568106),
 ('意味着', 0.026864608904568106),
 ('购买力', 0.026864608904568106),
 ('高桥洋', 0.026864608904568106)]

In [12]:
from spacy.lang.zh import Chinese

nlp = Chinese()
doc = nlp(u"蘋果公司正考量用一億元買下英國的新創公司，加上一個標點。")

In [13]:
[tk for tk in doc]

[蘋果, 公司, 正, 考量, 用, 一, 億元, 買, 下, 英國, 的, 新創, 公司, ，, 加上, 一個, 標點, 。]

# Han NLP

In [2]:
# spacy

# from spacy.lang.zh import Chinese

# Chinese.Defaults.use_jieba = False
# nlp = Chinese()

# # Disable jieba through tokenizer config options
# cfg = {"use_jieba": False}
# nlp = Chinese(meta={"tokenizer": {"config": cfg}})

# # Load with "default" model provided by pkuseg
# cfg = {"pkuseg_model": "default", "require_pkuseg": True}
# nlp = Chinese(meta={"tokenizer": {"config": cfg}})



✘ No compatible model found for 'zh_core_web_sm' (spaCy v2.2.3).



# abcnews (event clustering)

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv("/workspace/flair/data/abcnews-date-text.csv")
df = df.head(1000)
df.head

<bound method NDFrame.head of      publish_date                                      headline_text
0        20030219  aba decides against community broadcasting lic...
1        20030219     act fire witnesses must be aware of defamation
2        20030219     a g calls for infrastructure protection summit
3        20030219           air nz staff in aust strike for pay rise
4        20030219      air nz strike to affect australian travellers
..            ...                                                ...
995      20030224                conference to focus on tuna fishery
996      20030224                      council hosts farewell for mp
997      20030224                council resists eba roster pressure
998      20030224                   customs house restoration opened
999      20030224              dam water levels still critically low

[1000 rows x 2 columns]>

In [3]:
from flair.data import Sentence
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings, DocumentRNNEmbeddings

document_embeddings = DocumentPoolEmbeddings([WordEmbeddings('glove')])

X = []
for i, d in enumerate(df["headline_text"]):
    sent = Sentence(d, use_tokenizer=True)
    document_embeddings.embed(sent)
    x = sent.get_embedding()
    x = x.detach().numpy()
    X.append(x)
X = np.stack(X)

In [23]:
# 將doc vectors依照日期分成不同的組

# df.groupby(by="publish_date").agg(["count"])

groups = df.groupby(by="publish_date")
groups.groups.keys()
keys = [20030219, 20030220, 20030221, 20030222, 20030223, 20030224]


# groups[20030219]
# groups.get_group(20030219)
d0 = groups.get_group(keys[0])
d1 = groups.get_group(keys[1])
d2 = groups.get_group(keys[2])

# for i, d in enumerate(df["headline_text"]):
#     print(i, d)
#     break

X0 = X[d0.index[0]:d0.index[-1]+1]
X1 = X[d1.index[0]:d1.index[-1]+1]
# X2 = X[d2.index[0]:d2.index[-1]+1]
# X0_1 = X[d0.index[0]:d1.index[-1]+1]


In [8]:
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation, MeanShift, AgglomerativeClustering

clustering = KMeans(n_clusters=50, random_state=0).fit(X0)
# clustering.predict(X1)
# y = np.concatenate([c0.labels_, clustering.predict(X1)])
# c1 = KMeans(n_clusters=45, random_state=0).fit(X1)

# # z = np.zeros(df.shape[0], dtype=np.int32)
# z = np.empty((df.shape[0],)) * np.nan
# z[:c0.labels_.shape[0]] = c0.labels_
# df["c0"] = z

# z = np.empty((df.shape[0],)) * np.nan
# z[:c1.labels_.shape[0]] = c1.labels_
# df["c1"] = z
# df.head(100)

# _df = df[0:len(y)]
# _df["y"] = y
# # _df.loc[_df["y"] == 20, ["headline_text", "y"]]
# _df.groupby("y").agg(["count"])

In [22]:
df = pd.DataFrame({"a" : [4 ,5, 6]})
# df.insert(loc=1, column="b", value=[1,2])
c = np.full((3,), np.nan).astype(np.uint8)
c[1:3] = [1,2]
df["c"] = c
df

prev = [1,1,1,np.nan]
cur = [1,2,3,4]

cluster = {}
# docs = pd.DataFrame({"doc" : [})
df = pd.DataFrame({
    "prev" : ["t0-1" ,"t0-1", "t0-1","NA", "NA"], 
    "cur": [1,1,2,1,2]
}, index = [5, 6, 7, 8, 9])


# g = grouped.get_group(1)
# list(grouped.groups)
# grouped
# print(grouped.get_group(2))


# df = grouped.get_group(1)
# print(df)
# g = grouped.get_group(1).groupby('prev')

# g.apply(lambda x: )
# [c for c in g.count()]
# g.count()

# len(df)

# grouped.get_group(2)
# for g in grouped:
#     print(g.)

   
    
def set_cluster(df: pd.DataFrame, cur_cluster: int):
    count = {}

    for i, r in df.iterrows():
        try:
            count[r["prev"]].append(i)
        except KeyError:
            count[r["prev"]] = [i]

    # 找出最大的prev_cluster
    cmax = (None, 0)  # (prev_label, count)
    total = 0 # number of prev items
    for k, v in count.items():
        if len(v) > cmax[1]:
            cmax = (k, len(v))
        if k != -1:
            total += len(v)

    print(cmax)

    # 判斷該cluster是否為已建立event的延續
    for i, r in df.iterrows():
        c = cmax[0] if cmax[1] / total > 0.5 else f't1-{cur_cluster}'
        try:
            cluster[i].append(c)
        except KeyError:
            cluster[i] = [c]
            
    

# k = 1
# g = grouped.get_group(1)
# df.iloc(1)
# cluster[0].append(1)
# cluster.values()

grouped = df.groupby("cur")
for k, g in grouped:
    set_cluster(g, cur_cluster=k)

cluster

('t0-1', 2)
('t0-1', 1)


{5: ['t0-1'], 6: ['t0-1'], 8: ['t0-1'], 7: ['t1-2'], 9: ['t1-2']}

In [194]:
a = {"a": "b"}
a[1] = 1

In [60]:
# 查看cluster內容
clustering = KMeans(n_clusters=100, random_state=0).fit(X)

# len(d0)
# len(clustering.labels_)
# d0['cluster'] = [i for i in range(198)]
df["cluster"] = clustering.labels_
grouped = d0.groupby("cluster")
series = grouped.apply(lambda x: x["headline_text"].tolist())
# series.to_frame()
for x in series:
    print(x)

# for i, x in grouped:
#     x["headline_text"]
# def f(group):
#     return pd.DataFrame({'original': group, 'demeaned': })
 

['community urged to help homeless youth', 'more women urged to become councillors', 'unions to ask members to support public protests']
['gold coast to hear about bilby project', 'group to meet in north west wa over rock art', 'meeting to focus on broken hill water woes', 'mugabe to touch down in paris for summit', 'peace agreement may bring respite for venezuela', 'rice mill closures to put 300 out of work']
['investigations underway into death toll of korean', 'investigation underway into elster creek spill']
['cemeteries miss out on funds', 'de villiers to learn fate on march 5', 'dog mauls 18 month old toddler in nsw', 'england change three for wales match', 'griffiths under fire over project knock back', 'hanson should go back where she came from nsw mp', 'harrington raring to go after break', 'martin to lobby against losing nt seat in fed', 'more than 40 pc of young men drink alcohol at', 'mp raises hospital concerns in parliament', 'national gallery gets all clear after', 'oh b

In [37]:
groups

In [15]:
c0 = KMeans(n_clusters=50, random_state=0).fit(X0)
c0.labels_

array([23, 20, 18, 13, 13, 42, 26, 31, 39, 14,  5, 41, 18, 40, 32, 14, 36,
       47, 26, 36,  5, 31, 32, 41,  3, 23, 34,  0, 23, 43, 23, 28, 28, 36,
       11, 10, 36, 31,  3, 14, 36, 21,  3, 10,  3, 32, 23,  8, 17, 38, 34,
        5,  5,  5, 29, 29, 25, 46, 38,  1,  6, 36, 21,  3,  1, 34, 41,  3,
        3, 21, 21, 27, 22,  2,  2, 36,  5,  7, 21, 19, 25, 45, 15, 32, 10,
       26, 13,  7,  7,  9,  7, 48,  3, 21, 33, 32,  1, 11,  3, 21,  0, 43,
        3,  4,  1,  3, 23,  9, 14, 21, 14, 14, 37, 13, 35, 30,  3,  3, 12,
       23, 41, 14, 23, 25, 23, 36, 41,  1, 44, 18, 32,  3, 38,  9, 20, 24,
       21, 32, 20, 13, 13, 14, 10, 49, 15, 37,  3, 34,  7,  8, 21, 34, 36,
       36, 49,  1, 36, 49,  3, 36, 47, 10, 39, 41, 19, 11,  6,  7, 32, 34,
       41, 10,  3, 32, 13, 41, 41, 36, 10,  3, 25, 16, 49,  0, 21, 47, 21,
        3, 43,  3, 41, 49, 40, 41, 23, 26,  3, 15], dtype=int32)

In [205]:
gs = df.groupby("c0")
df.loc[df["c1"] == 2, ["headline_text", "c0", "c1"]]

,headline_text,c0,c1
22,call for ethanol blend fuel to go ahead,18.0,2.0
69,health minister backs organ and tissue storage,5.0,2.0
85,last minute call hands alinghi big lead,22.0,2.0
100,more women urged to become councillors,20.0,2.0
117,omodei to stay in politics,28.0,2.0
145,rain eases wheatbelt water woes,17.0,2.0
184,uni to continue tree disease study,20.0,2.0
189,vowles to retire at end of season,28.0,2.0
215,brisbane sparkies head for 10 day strike,NaN,2.0
219,bungle leaves doctor waiting to practise,NaN,2.0


In [189]:
a = np.array([1,2,3])
z = np.zeros(5, dtype=np.int16)
z[:a.shape[0]] = a
# z
# a.shape
# a.shape[0]
z

array([1, 2, 3, 0, 0], dtype=int16)

In [87]:
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation, MeanShift, AgglomerativeClustering

# clustering = KMeans(n_clusters=100, random_state=0).fit(X)
# clustering = DBSCAN(eps=0.11, min_samples=1, metric="cosine").fit(X)
# clustering = AffinityPropagation(max_iter=2000, convergence_iter=15,).fit(X)
# clustering = MeanShift(max_iter=3000).fit(X)
# clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=3).fit(X)

print(clustering.labels_)
df["y"] = list(clustering.labels_)

# print(kmeans.predict([[0, 0], [12, 3]]))
# print(kmeans.cluster_centers_)
# df["label"] = kmeans.labels_
# df[df["label"] == 0]

# len(df)
df.loc[df["y"] == 1, ["headline_text", "y"]]

[ 82   0 120  91  91  65  26 109  21 115 212 148 234 225  30 107  73 105
   3  75   0 153  49 148 213  36  40 132 114 260  15 229 229 159  33  90
   9 109 135  93 159  47  23 168  42 219  76  76 156  24 170  84  84 120
 254 254 205  70  24 193  45 159 151 154  15 212  12 115  79  36  48 169
 214  90 161   9 120  52 136  62 205  60 185 130 168 100  91  52   8 203
   8 166 176 121 244  76 120 171 152  30 132 260  73 150 135 115 183 203
  87  25 213  32  39 190  10 207 240 135 122 132 181  41  19 106  19 217
 126 226 259 120 128 154  24 203  80  18 235  16   6  91 234  82   6  81
 145 129  61 178 223  47 237 111 206 118 119  55  56  81 100 137  64  90
 155  12  62 171  45  34  49  16 165 152  41 219  28  41 122  22  15  70
 106 204 124 132  25 105  16 260 139  29  26 215 227  58 186 201 141  14
 257 100 162 114 176 170 175  79  96  82 124 234 157  13 148  96  93  29
  64  10 141 256 117 183 120  27 252 101 201  67 251  40  46  19  46 200
   0 131  81  97 114  76  41 118 160  51  77 189  9

,headline_text,y
735,former star koen licks cats,1
842,cat lovers march for romes sacred strays,1
844,cats claw dogs pies swoop on blues,1
866,fake crocs scare off flamingos,1
945,tourists cry foul over spilt milk,1


# Pandas

In [159]:
import pandas as pd

df = pd.DataFrame({"a" : [1,2,3,4,5],"b" : [6, 7, 8, 9,10]})
# df["d"] = None
# df.loc[0, "d"] = 1
# df["a"].tolist()
# df.loc[0:2, "c"] = [0,1]
# df["c"] = None
# df.loc[0:2, "c"] = [0,1,2]
df["c"] = np.zeros(5).astype(np.int16)
df
# df["c"].apply(pd.to_numeric)
a = np.zeros(5).astype(np.int16)
a[0:3] = np.array([1,2,3])
a

# groups = df.groupby(by="publish_date")
# groups.iloc['20030219']
# .agg(["count"])
# for name, group in groups:
#     print(name)
#     for i, row in group.iterrows():
#         print(row['headline_text'])
#         col = row['column']
#         column_type = row['column_type']
#         is_null = 'NOT NULL' if row['is_null'] == 'NO' else ''
#         print('\t{} {} {},'.format(col, column_type, is_null))

# gs

array([1, 2, 3, 0, 0], dtype=int16)

# ???

In [34]:
from pprint import pprint
import os 
import json 
directory = "/workspace/stocknet-dataset/tweet/raw"

for ticker in os.listdir(directory):
    p = os.path.join(directory, ticker)
    if os.path.isfile(p):
        continue
    for fn in os.listdir(p):
        with open(os.path.join(directory, ticker, fn)) as f:
            for i, l in enumerate(f.readlines()):
                print(i)
                j = json.loads(l)
#                 print(j["text"])
                pprint(j)
#                 print(l)
        break
    break

# [f for f in listdir(p)]
# for root, dirnames, filenames in os.walk(d):
#     for fileanme in filenames:
#         p = os.path.join(root, dirname, filename)
#         print(p)
#         with open(p) as f:
#             pass
#         break
        
#     for filename in filenames:
#         if ".DS_Store" in filename:
#             continue
#         path = os.path.join(root, filename)
#         print(root, filename)
#         break


0
{'contributors': None,
 'coordinates': None,
 'created_at': 'Tue Oct 14 13:39:49 +0000 2014',
 'entities': {'hashtags': [],
              'symbols': [{'indices': [0, 4], 'text': 'GDP'},
                          {'indices': [42, 47], 'text': 'TSLA'},
                          {'indices': [48, 53], 'text': 'TWTR'},
                          {'indices': [54, 56], 'text': 'C'},
                          {'indices': [57, 61], 'text': 'GDP'},
                          {'indices': [62, 67], 'text': 'AMZN'},
                          {'indices': [68, 73], 'text': 'DRYS'},
                          {'indices': [74, 77], 'text': 'VZ'}],
              'trends': [],
              'urls': [{'display_url': 'twitter.com/CBOE/status/52…',
                        'expanded_url': 'https://twitter.com/CBOE/status/522018942626594816',
                        'indices': [81, 104],
                        'url': 'https://t.co/7Q6GX3gUuS'},
                       {'display_url': 'quantpost.com/list/?symbo

In [1]:
import pandas as pd
from gensim.parsing.preprocessing import remove_stopwords

df = pd.read_csv("/workspace/flair/data/Full-Economic-News-DFE-839861.csv", encoding='ISO-8859-1')
# documents = df.loc[:100, "text"].tolist()
df = df.loc[:10, ["headline"]]
df['label'] = [i for i in range(0,11)]
df

,headline,label
0,Yields on CDs Fell in the Latest Week,0
1,The Morning Brief: White House Seeks to Limit ...,1
2,Banking Bill Negotiators Set Compromise --- Pl...,2
3,Manager's Journal: Sniffing Out Drug Abusers I...,3
4,Currency Trading: Dollar Remains in Tight Rang...,4
5,"Stocks Fall Again; BofA, Alcoa Slide",5
6,"Bank of Montreal, Royal Bank Profits Rose in 2...",6
7,Battle Over Medical Costs Isn't Over,7
8,Sequenom to Buy Gemini Genomics In Stock Accord,8
9,U.S. Dollar Falls Against Most Currencies; Dec...,9


In [55]:
# documents
df = pd.DataFrame({"d" : ["aaa" ,"bbb", "ccc"]})
df['label'] = [1,1,0]
df[df.label == 1]

[x for x in df["d"].head(1)]

['aaa']

# flair

In [5]:
from flair.data import Sentence
from flair.models import SequenceTagger

# make a sentence
sentence = Sentence('I love Berlin .')

# load the NER tagger
tagger = SequenceTagger.load('ner')

# run NER over sentence
tagger.predict(sentence)

2020-07-04 15:40:02,350 loading file /root/.flair/models/en-ner-conll03-v0.4.pt


In [6]:
from sklearn.cluster import KMeans
import numpy as np
X = np.array([[1, 2], [1, 4], [1, 0], [10, 2], [10, 4], [10, 0]])
# X.shape
# kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
# print(kmeans.labels_)
# print(kmeans.predict([[0, 0], [12, 3]]))
# print(kmeans.cluster_centers_)

In [7]:
import pandas as pd

df = pd.read_csv("/workspace/flair/data/Full-Economic-News-DFE-839861.csv", encoding='ISO-8859-1')
df = df.head(100)
# [d for d in df["headline"]]
# df.columns
g = df.groupby(["date"])
g.get_group('11/23/11')
# s = g.size()
# s[s > 1]

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,articleid,date,headline,positivity_gold,relevance_gold,text
5,842613460,False,finalized,3,12/4/15 23:15,3.0,0.6783,yes,1.0,wsj_905654974,11/23/11,"Stocks Fall Again; BofA, Alcoa Slide",NaN,NaN,"Stocks declined, as investors weighed slower-t..."
56,842613511,False,finalized,3,12/5/15 7:26,NaN,NaN,no,1.0,wsj_905646207,11/23/11,Indian Rupee Falls Again,NaN,NaN,MUMBAI - The Indian rupee fell against the U.S...


In [8]:
import pandas as pd

df = pd.read_csv("/workspace/flair/data/abcnews-date-text.csv")
df = df.head(2000)

In [9]:
g = df.groupby(["publish_date"])
g.size()

publish_date
20030219    198
20030220    250
20030221    250
20030222    126
20030223    136
20030224    250
20030225    250
20030226    250
20030227    221
20030228     69
dtype: int64

In [11]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from flair.data import Sentence
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings, DocumentRNNEmbeddings

df = pd.read_csv("/workspace/flair/data/Full-Economic-News-DFE-839861.csv", encoding='ISO-8859-1')
df = df.head(100)

document_embeddings = DocumentPoolEmbeddings([WordEmbeddings('glove')])

X = []
for d in df["headline"]:
    sent = Sentence(d, use_tokenizer=True)
    document_embeddings.embed(sent)
    x = sent.get_embedding()
    x = x.detach().numpy()
#     print(x)
    X.append(x)

X = np.stack(X)
kmeans = KMeans(n_clusters=10, random_state=0).fit(X)
# print(len(kmeans.labels_))
df["y"] = list(kmeans.labels_)
# print(kmeans.predict([[0, 0], [12, 3]]))
# print(kmeans.cluster_centers_)
# df["label"] = kmeans.labels_
# df[df["label"] == 0]

# len(df)
df.loc[df["y"] == 1, ["headline", "y"]]

,headline,y
4,Currency Trading: Dollar Remains in Tight Rang...,1
9,U.S. Dollar Falls Against Most Currencies; Dec...,1
17,Dollar Declines as Players Take Profits From R...,1
26,Bond Prices Tumble as Dollar's Plunge Prompts ...,1
35,"Tuesday's markets: Prices of bonds, stocks dro...",1
47,Retailers Stock Up on Caution,1
48,Some Stocks Had Big Fourth-Quarter Gains Despi...,1
52,Bond Prices Are Little Changed In Slow Trading...,1
60,Futures Markets: Treasury Bond Contracts End N...,1
62,"AMR, Southwest Profits Surge Even as Fuel Pric...",1


In [13]:
X = np.stack(X)
kmeans = KMeans(n_clusters=10, random_state=0).fit(X)
# print(len(kmeans.labels_))
df["y"] = list(kmeans.labels_)
# print(kmeans.predict([[0, 0], [12, 3]]))
# print(kmeans.cluster_centers_)
# df["label"] = kmeans.labels_
# df[df["label"] == 0]

# len(df)
df.loc[df["y"] == 1, ["headline", "y"]]

,headline,y
4,Currency Trading: Dollar Remains in Tight Rang...,1
9,U.S. Dollar Falls Against Most Currencies; Dec...,1
17,Dollar Declines as Players Take Profits From R...,1
26,Bond Prices Tumble as Dollar's Plunge Prompts ...,1
35,"Tuesday's markets: Prices of bonds, stocks dro...",1
47,Retailers Stock Up on Caution,1
48,Some Stocks Had Big Fourth-Quarter Gains Despi...,1
52,Bond Prices Are Little Changed In Slow Trading...,1
60,Futures Markets: Treasury Bond Contracts End N...,1
62,"AMR, Southwest Profits Surge Even as Fuel Pric...",1


[7, 9, 1, 0, 2, 8, 3, 5, 6, 2, 4]

In [97]:
# from flair.data import Sentence
from flair.embeddings import Sentence, WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings, DocumentRNNEmbeddings

sentence = Sentence('this is one. this is two. this is three.', use_tokenizer=True)
# sentence.tokens
# now check out the embedded tokens.
# for token in sentence:
#     print(token)
#     print(token.embedding)

# document_embeddings = DocumentPoolEmbeddings([WordEmbeddings('glove'),
#                                               FlairEmbeddings('news-backward'),
#                                               FlairEmbeddings('news-forward')])
document_embeddings = DocumentPoolEmbeddings([WordEmbeddings('glove')])
document_embeddings.embed(sentence)
# print(sentence.get_embedding())

# from flair.embeddings import WordEmbeddings, DocumentRNNEmbeddings

# glove_embedding = WordEmbeddings('glove')
# document_embeddings = DocumentRNNEmbeddings([glove_embedding])
# sentence = Sentence('The grass is green . And the sky is blue .')
# document_embeddings.embed(sentence)
# print(sentence.get_embedding())

# document_lstm_embeddings = DocumentRNNEmbeddings([WordEmbeddings('glove')], rnn_type='LSTM')
# document_embeddings.embed(sentence)
# print(sentence.get_embedding())

x = sentence.get_embedding()
x.detach().numpy()

(100,)

In [2]:
from flair.data import Sentence

# sent = Sentence('The grass is green.', use_tokenizer=True)
sent = Sentence('The grass is green.')

In [17]:
from flair.data import Corpus
from flair.datasets import ClassificationCorpus

corpus: Corpus = ClassificationCorpus('/workspace/flair/data/@me_dataset',
                                      test_file='test.txt',
                                      dev_file='dev.txt',
                                      train_file='train.txt')


2020-07-04 15:46:44,343 Reading data from /workspace/flair/data/@me_dataset
2020-07-04 15:46:44,352 Train: /workspace/flair/data/@me_dataset/train.txt
2020-07-04 15:46:44,359 Dev: /workspace/flair/data/@me_dataset/dev.txt
2020-07-04 15:46:44,363 Test: /workspace/flair/data/@me_dataset/test.txt


In [22]:
from flair.data import segtok_tokenizer, Sentence
from flair.datasets import ClassificationDataset
from flair.datasets import ClassificationCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

# ds = ClassificationDataset(
#     path_to_file='/workspace/flair/data/news/train.txt',
#     in_memory=True,
# )
# ds[0]

# corpus = ClassificationCorpus(data_folder='/workspace/flair/data/news', tokenizer=segtok_tokenizer, in_memory=True)
# corpus.train[0]
corpus: Corpus = ClassificationCorpus('/workspace/flair/data/@me_dataset',
                                      test_file='test.txt',
                                      dev_file='dev.txt',
                                      train_file='train.txt',
                                     tokenizer=segtok_tokenizer)
label_dict = corpus.make_label_dictionary()

# document_embeddings = DocumentPoolEmbeddings([WordEmbeddings('glove')])
# word_embeddings = [WordEmbeddings('glove'),
#                    # comment in flair embeddings for state-of-the-art results
#                    # FlairEmbeddings('news-forward'),
#                    # FlairEmbeddings('news-backward'),
#                    ]
# document_embeddings = DocumentPoolEmbeddings([WordEmbeddings('glove'),
# #                    FlairEmbeddings('news-forward'),
# #                    FlairEmbeddings('news-backward'),
# #                                              ])
# document_embeddings(sent)

document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)

classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)
trainer = ModelTrainer(classifier, corpus)
trainer.train('resources/taggers/test',
              learning_rate=0.1,
              mini_batch_size=32,
              anneal_factor=0.5,
              patience=5)

2020-07-04 15:51:44,909 Reading data from /workspace/flair/data/@me_dataset
2020-07-04 15:51:44,918 Train: /workspace/flair/data/@me_dataset/train.txt
2020-07-04 15:51:44,933 Dev: /workspace/flair/data/@me_dataset/dev.txt
2020-07-04 15:51:44,951 Test: /workspace/flair/data/@me_dataset/test.txt
2020-07-04 15:51:45,040 Computing label dictionary. Progress:


100%|██████████| 2/2 [00:00<00:00,  3.22it/s]

2020-07-04 15:51:46,167 [b'aaa', b'bbb']
2020-07-04 15:51:46,227 ----------------------------------------------------------------------------------------------------
2020-07-04 15:51:46,242 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
    )
    (word_reprojection_map): Linear(in_features=100, out_features=256, bias=True)
    (rnn): GRU(256, 512, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=512, out_features=2, bias=True)
  (loss_function): BCEWithLogitsLoss()
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2020-07-04 15:51:46,247 ----------------------------------------------------------------------------------------------------
2020-07-04 15:51:46,259 Corpus: "Corpus: 1 train + 1 dev + 1 test sentences"
2020-07-04 15:51:46,276 ------------------------------------------------------------------------------------

2020-07-04 15:51:46,377  - shuffle: "True"
2020-07-04 15:51:46,407  - train_with_dev: "False"
2020-07-04 15:51:46,414  - batch_growth_annealing: "False"
2020-07-04 15:51:46,423 ----------------------------------------------------------------------------------------------------
2020-07-04 15:51:46,443 Model training base path: "resources/taggers/test"
2020-07-04 15:51:46,458 ----------------------------------------------------------------------------------------------------
2020-07-04 15:51:46,464 Device: cpu
2020-07-04 15:51:46,532 ----------------------------------------------------------------------------------------------------
2020-07-04 15:51:46,548 Embeddings storage mode: cpu
2020-07-04 15:51:46,601 ----------------------------------------------------------------------------------------------------
2020-07-04 15:51:47,692 epoch 1 - iter 1/1 - loss 0.83057070 - samples/sec: 62.67
2020-07-04 15:51:49,305 -----------------------------------------------------------------------------

EOFError: Ran out of input

In [16]:
document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)

classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)
trainer = ModelTrainer(classifier, corpus)
trainer.train('resources/taggers/ag_news',
              learning_rate=0.1,
              mini_batch_size=32,
              anneal_factor=0.5,
              patience=5)

# from flair.visual.training_curves import Plotter
# plotter = Plotter()
# plotter.plot_weights('resources/taggers/ag_news/weights.txt')


# classifier = TextClassifier.load('resources/taggers/ag_news/final-model.pt')
# sentence = Sentence('France is the current world cup winner.')
# classifier.predict(sentence)
# print(sentence.labels)

NameError: name 'label_dict' is not defined

In [10]:
from flair.data import Sentence
from flair.embeddings import WordEmbeddings

sent = Sentence('The grass is green.', use_tokenizer=True)
# glove = WordEmbeddings('glove')
# glove.embed(sent)

# # now check out the embedded tokens.
# for token in sentence:
#     print(token)
#     print(token.embedding)

# [x.embedding.shape for x in sent]
# [x for x in sent]
sent

Sentence: "The grass is green ."   [− Tokens: 5]

# gensim

In [41]:
import pandas as pd
from gensim.parsing.preprocessing import remove_stopwords

df = pd.read_csv("/workspace/flair/data/Full-Economic-News-DFE-839861.csv", encoding='ISO-8859-1')
documents = df.loc[:1000, "text"].tolist()
# docs[0]
# documents = [
#     "Human machine interface for lab abc computer applications",
#     "A survey of user opinion of computer system response time",
#     "The EPS user interface management system",
#     "System and human system engineering testing of EPS",
#     "Relation of user perceived response time to error measurement",
#     "The generation of random binary unordered trees",
#     "The intersection graph of paths in trees",
#     "Graph minors IV Widths of trees and well quasi ordering",
#     "Graph minors A survey",
# ]

from pprint import pprint  # pretty-printer
from collections import defaultdict

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in remove_stopwords(d).lower().split()]
    for d in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

# pprint(texts)

from gensim import corpora

dictionary = corpora.Dictionary(texts)
# dictionary.save('/tmp/deerwester.dict')  # store the dictionary, for future reference
# print(dictionary)
corpus = [dictionary.doc2bow(text) for text in texts]
# corpus

In [44]:
from gensim import models

tfidf = models.TfidfModel(corpus)  # step 1 -- initialize a model
corpus_tfidf = tfidf[corpus]
# for doc in corpus_tfidf:
#     print(doc)

# lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=3)  # initialize an LSI transformation
model = models.LdaModel(corpus, id2word=dictionary, num_topics=10)
corpus_lsi = model[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
model.print_topics(10)

# both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
# for doc, as_text in zip(corpus_lsi, documents):
#     print(doc, as_text)


[(0,
  '0.011*"--" + 0.009*"the" + 0.007*"u.s." + 0.006*"said" + 0.005*"federal" + 0.004*"mr." + 0.004*"market" + 0.004*"fed" + 0.004*"economic" + 0.003*"new"'),
 (1,
  '0.011*"the" + 0.009*"new" + 0.007*"market" + 0.007*"said" + 0.007*"u.s." + 0.005*"mr." + 0.005*"stock" + 0.005*"fed" + 0.005*"economic" + 0.005*"--"'),
 (2,
  '0.008*"--" + 0.007*"new" + 0.007*"u.s." + 0.005*"market" + 0.005*"said" + 0.005*"the" + 0.004*"stock" + 0.003*"economic" + 0.003*"billion" + 0.003*"rates"'),
 (3,
  '0.009*"the" + 0.007*"new" + 0.006*"stock" + 0.006*"u.s." + 0.006*"prices" + 0.005*"said" + 0.004*"market" + 0.004*"economic" + 0.004*"--" + 0.004*"companies"'),
 (4,
  '0.011*"--" + 0.009*"the" + 0.007*"said" + 0.007*"new" + 0.007*"u.s." + 0.006*"market" + 0.006*"stock" + 0.005*"economic" + 0.004*"rates" + 0.004*"york"'),
 (5,
  '0.010*"the" + 0.009*"u.s." + 0.008*"--" + 0.006*"said" + 0.006*"stock" + 0.006*"investors" + 0.005*"market" + 0.005*"new" + 0.005*"year" + 0.004*"rates"'),
 (6,
  '0.009*"s

In [18]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_lg")  # make sure to use larger model!

dog dog 1.0
dog cat 0.80168545
dog banana 0.24327646
cat dog 0.80168545
cat cat 1.0
cat banana 0.2815437
banana dog 0.24327646
banana cat 0.2815437
banana banana 1.0


In [27]:
# F,Ford Motor Company,NYQ,Auto Manufacturers - Major,USA,,,
# MSFT,Microsoft Corporation,NMS,Business Software & Services,USA,,,
tks = nlp("AXP American Express Company")
# t1, t2 = nlp("F Ford")

# for token in tks:
#     print(token.text, token.has_vector, token.vector_norm, token.is_oov)

# for token1 in tks:
token1 = tks[0]
for token2 in tks[1:]:
    print(token1.text, token2.text, token1.similarity(token2))

AXP American 0.041551307
AXP Express 0.011548235
AXP Company -0.17779963


In [40]:
t1 = "The Walt Disney Company"
t2 = "The Walt Disney"
d1 = nlp(t1)
d2 = nlp(t2)

# type(doc.ents[0])
# displacy.render(doc, style="ent")
# s = doc.ents[0]
# s.vector
# d1.similarity(d2)


The Walt Disney Company


In [2]:
import spacy


t = 'NEW YORK -- Yields on most certificates of deposit offered by major banks dropped more than a tenth of a percentage point in the latest week, reflecting the overall decline in short-term interest rates. On small-denomination, or "consumer," CDs sold directly by banks, the average yield on six-month deposits fell to 5.49% from 5.62% in the week ended yesterday, according to an 18-bank survey by Banxquote Money Markets, a Wilmington, Del., information service.</br></br>On three-month "consumer" deposits, the average yield sank to 5.29% from 5.42% the week before, according to Banxquote. Two banks in the Banxquote survey, Citibank in New York and CoreStates in Pennsylvania, are paying less than 5% on threemonth small-denomination CDs.</br></br>Declines were somewhat smaller on five-year consumer CDs, which eased to 7.37% from 7.45%, Banxquote said.</br></br>Yields on three-month and six-month Treasury bills sold at Monday\'s auction plummeted more than a fifth of a percentage point from the previous week, to 5.46% and 5.63%, respectively.'

nlp = spacy.load("en_core_web_sm")
doc = nlp(t)

In [6]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
displacy.render(doc, style="ent")
doc[0]
doc[0].text, doc[0].ent_type_, doc[0].ent_kb_id_

In [1]:
from flair.data import Corpus
from flair.datasets import TREC_6
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

corpus = TREC_6()
label_dict = corpus.make_label_dictionary()
word_embeddings = [WordEmbeddings('glove'),
                   # comment in flair embeddings for state-of-the-art results
                   FlairEmbeddings('news-forward'),
                   # FlairEmbeddings('news-backward'),
                   ]
document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)
trainer = ModelTrainer(classifier, corpus)

trainer.train('resources/taggers/ag_news',
              learning_rate=0.1,
              mini_batch_size=32,
              anneal_factor=0.5,
              patience=5,
              max_epochs=150)

from flair.visual.training_curves import Plotter
plotter = Plotter()
plotter.plot_weights('resources/taggers/ag_news/weights.txt')

2020-03-11 14:01:10,984 Reading data from /root/.flair/datasets/trec_6
2020-03-11 14:01:10,987 Train: /root/.flair/datasets/trec_6/train.txt
2020-03-11 14:01:11,001 Dev: None
2020-03-11 14:01:11,007 Test: /root/.flair/datasets/trec_6/test.txt
2020-03-11 14:01:11,864 Computing label dictionary. Progress:


100%|██████████| 4907/4907 [00:00<00:00, 21493.21it/s]

2020-03-11 14:01:12,101 [b'LOC', b'ENTY', b'HUM', b'NUM', b'ABBR', b'DESC']


ValueError: cannot reshape array of size 417659 into shape (400000,100)

In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load('en-sentiment')
sentence = Sentence('Flair is pretty neat!')

classifier.predict(sentence)
# print sentence with predicted labels

print('Sentence above is: ', sentence.labels)

In [4]:
from flair.data import Sentence
from flair.models import SequenceTagger

# t = 'NEW YORK -- Yields on most certificates of deposit offered by major banks dropped more than a tenth of a percentage point in the latest week, reflecting the overall decline in short-term interest rates.</br></br>On small-denomination, or "consumer," CDs sold directly by banks, the average yield on six-month deposits fell to 5.49% from 5.62% in the week ended yesterday, according to an 18-bank survey by Banxquote Money Markets, a Wilmington, Del., information service.</br></br>On three-month "consumer" deposits, the average yield sank to 5.29% from 5.42% the week before, according to Banxquote. Two banks in the Banxquote survey, Citibank in New York and CoreStates in Pennsylvania, are paying less than 5% on threemonth small-denomination CDs.</br></br>Declines were somewhat smaller on five-year consumer CDs, which eased to 7.37% from 7.45%, Banxquote said.</br></br>Yields on three-month and six-month Treasury bills sold at Monday\'s auction plummeted more than a fifth of a percentage point from the previous week, to 5.46% and 5.63%, respectively.'
t = "iTV Will Boost Apple http:\/\/t.co\/8dup4cQc08 $AAPL #APPLE"
t = t.replace("AAPL", "")

sentence = Sentence(t)
# [x for x in sentence]

tagger = SequenceTagger.load("chunk-fast")
# tagger = SequenceTagger.load("ner-ontonotes-fast")
tagger.predict(sentence)

# print("Analysing %s" % sentence)
print(sentence.to_tagged_string())

2020-03-11 13:28:36,083 loading file /root/.flair/models/en-chunk-conll2000-fast-v0.4.pt
iTV <S-NP> Will <B-VP> Boost <E-VP> Apple <B-NP> http:\/\/t.co\/8dup4cQc08 <I-NP> $ <I-NP> #APPLE <E-NP>


In [5]:
sentence.get_spans('np')

[<NP-span (1): "iTV">,
 <VP-span (2,3): "Will Boost">,
 <NP-span (4,5,6,7): "Apple http:\/\/t.co\/8dup4cQc08 $ #APPLE">]

In [4]:
from flair.data import Sentence
from flair.models import SequenceTagger

t = 'NEW YORK -- Yields on most certificates of deposit offered by major banks dropped more than a tenth of a percentage point in the latest week, reflecting the overall decline in short-term interest rates.</br></br>On small-denomination, or "consumer," CDs sold directly by banks, the average yield on six-month deposits fell to 5.49% from 5.62% in the week ended yesterday, according to an 18-bank survey by Banxquote Money Markets, a Wilmington, Del., information service.</br></br>On three-month "consumer" deposits, the average yield sank to 5.29% from 5.42% the week before, according to Banxquote. Two banks in the Banxquote survey, Citibank in New York and CoreStates in Pennsylvania, are paying less than 5% on threemonth small-denomination CDs.</br></br>Declines were somewhat smaller on five-year consumer CDs, which eased to 7.37% from 7.45%, Banxquote said.</br></br>Yields on three-month and six-month Treasury bills sold at Monday\'s auction plummeted more than a fifth of a percentage point from the previous week, to 5.46% and 5.63%, respectively.'

tagger = SequenceTagger.load("chunk-fast")
sentence = Sentence(t)
tagger.predict(sentence)
sentence.to_tagged_string()

2020-02-28 06:10:34,080 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/models-v0.4/release-chunk-fast-0/en-chunk-conll2000-fast-v0.4.pt not found in cache, downloading to /tmp/tmpdhmzfc8v


100%|██████████| 75233247/75233247 [00:37<00:00, 2018218.86B/s]

2020-02-28 06:11:13,724 copying /tmp/tmpdhmzfc8v to cache at /root/.flair/models/en-chunk-conll2000-fast-v0.4.pt


2020-02-28 06:11:13,899 removing temp file /tmp/tmpdhmzfc8v
2020-02-28 06:11:13,930 loading file /root/.flair/models/en-chunk-conll2000-fast-v0.4.pt


'NEW <B-NP> YORK <E-NP> -- Yields <S-NP> on <S-PP> most <B-NP> certificates <E-NP> of <S-PP> deposit <S-NP> offered <S-VP> by <S-PP> major <B-NP> banks <E-NP> dropped <S-VP> more <B-NP> than <I-NP> a <I-NP> tenth <E-NP> of <S-PP> a <B-NP> percentage <I-NP> point <E-NP> in <S-PP> the <B-NP> latest <I-NP> week, <E-NP> reflecting <S-VP> the <B-NP> overall <I-NP> decline <E-NP> in <S-PP> short-term <B-NP> interest <I-NP> rates.</br></br>On <E-NP> small-denomination, or "consumer," <B-NP> CDs <E-NP> sold <S-VP> directly <S-ADVP> by <S-PP> banks, <S-NP> the <B-NP> average <I-NP> yield <E-NP> on <S-PP> six-month <B-NP> deposits <E-NP> fell <S-VP> to <S-PP> 5.49% <S-NP> from <S-PP> 5.62% <S-NP> in <S-PP> the <B-NP> week <E-NP> ended <S-VP> yesterday, <S-NP> according <S-PP> to <S-PP> an <B-NP> 18-bank <I-NP> survey <E-NP> by <S-PP> Banxquote <B-NP> Money <I-NP> Markets, <E-NP> a <B-NP> Wilmington, <I-NP> Del., <I-NP> information <I-NP> service.</br></br>On <I-NP> three-month <I-NP> "consumer" 

In [5]:
for entity in sentence.get_spans('ner'):
    print(entity)

In [4]:
import pandas as pd

df = pd.read_csv("/workspace/flair/data/Full-Economic-News-DFE-839861.csv", encoding='ISO-8859-1')
# df.head
# _unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,articleid,date,headline,positivity_gold,relevance_gold,text
# a, b, c = df.loc[0, ["date", "headline", "text"]]
# df.loc[12, ["date", "headline", "text"]][1]

'Dow Falls 45.95, Late GM Surge Stanches Losses'

In [23]:
# df.loc[10, ["date", "headline", "text"]][2]
docs = df.loc[:10, "text"].tolist()
docs[0]

'NEW YORK -- Yields on most certificates of deposit offered by major banks dropped more than a tenth of a percentage point in the latest week, reflecting the overall decline in short-term interest rates.</br></br>On small-denomination, or "consumer," CDs sold directly by banks, the average yield on six-month deposits fell to 5.49% from 5.62% in the week ended yesterday, according to an 18-bank survey by Banxquote Money Markets, a Wilmington, Del., information service.</br></br>On three-month "consumer" deposits, the average yield sank to 5.29% from 5.42% the week before, according to Banxquote. Two banks in the Banxquote survey, Citibank in New York and CoreStates in Pennsylvania, are paying less than 5% on threemonth small-denomination CDs.</br></br>Declines were somewhat smaller on five-year consumer CDs, which eased to 7.37% from 7.45%, Banxquote said.</br></br>Yields on three-month and six-month Treasury bills sold at Monday\'s auction plummeted more than a fifth of a percentage po

In [2]:
from flair.data import Sentence
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger.load("ner")

sentence: Sentence = Sentence("George Washington went to Washington .")
tagger.predict(sentence)

print("Analysing %s" % sentence)
print("\nThe following NER tags are found: \n")
print(sentence.to_tagged_string())

2020-02-26 13:30:40,873 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/models-v0.4/NER-conll03-english/en-ner-conll03-v0.4.pt not found in cache, downloading to /tmp/tmp4xhqochp


100%|██████████| 432197603/432197603 [47:36<00:00, 151296.68B/s] 

2020-02-26 14:18:18,476 copying /tmp/tmp4xhqochp to cache at /root/.flair/models/en-ner-conll03-v0.4.pt


2020-02-26 14:18:19,110 removing temp file /tmp/tmp4xhqochp
2020-02-26 14:18:19,240 loading file /root/.flair/models/en-ner-conll03-v0.4.pt
Analysing Sentence: "George Washington went to Washington ." - 6 Tokens

The following NER tags are found: 

George <B-PER> Washington <E-PER> went to Washington <S-LOC> .
